## Code Refrence : https://github.com/fawazsammani/chatbot-transformer/blob/master/transformer%20chatbot.ipynb

In [1]:
from collections import Counter
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.utils.data
import math
import torch.nn.functional as F

In [2]:
!wget http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip

--2023-12-20 03:33:11--  http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9916637 (9.5M) [application/zip]
Saving to: 'cornell_movie_dialogs_corpus.zip'

cornell_movie_dialo 100%[===================>]   9.46M  4.62MB/s    in 2.0s    

2023-12-20 03:33:14 (4.62 MB/s) - 'cornell_movie_dialogs_corpus.zip' saved [9916637/9916637]



In [3]:
!pwd

/kaggle/working


In [4]:
!unzip -o cornell_movie_dialogs_corpus.zip 

Archive:  cornell_movie_dialogs_corpus.zip
   creating: cornell movie-dialogs corpus/
  inflating: cornell movie-dialogs corpus/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/cornell movie-dialogs corpus/
  inflating: __MACOSX/cornell movie-dialogs corpus/._.DS_Store  
  inflating: cornell movie-dialogs corpus/chameleons.pdf  
  inflating: __MACOSX/cornell movie-dialogs corpus/._chameleons.pdf  
  inflating: cornell movie-dialogs corpus/movie_characters_metadata.txt  
  inflating: cornell movie-dialogs corpus/movie_conversations.txt  
  inflating: cornell movie-dialogs corpus/movie_lines.txt  
  inflating: cornell movie-dialogs corpus/movie_titles_metadata.txt  
  inflating: cornell movie-dialogs corpus/raw_script_urls.txt  
  inflating: cornell movie-dialogs corpus/README.txt  
  inflating: __MACOSX/cornell movie-dialogs corpus/._README.txt  


In [5]:
corpus_movie_conv = 'cornell movie-dialogs corpus/movie_conversations.txt'
corpus_movie_lines = 'cornell movie-dialogs corpus/movie_lines.txt'
max_len = 25

In [6]:
with open(corpus_movie_conv, 'r') as c:
    conv = c.readlines()

In [7]:
with open(corpus_movie_lines, 'r',encoding='iso-8859-1') as l:
    lines = l.readlines()


In [8]:
lines_dic = {}
for line in lines:
    objects = line.split(" +++$+++ ")
    lines_dic[objects[0]] = objects[-1]

In [9]:
first_element = next(iter(lines_dic))

In [10]:
print(lines_dic[first_element])

They do not!



In [11]:
def remove_punc(string):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in string:
        if char not in punctuations:
            no_punct = no_punct + char  # space is also a character
    return no_punct.lower()

In [12]:
pairs = []
for con in conv:
    ids = eval(con.split(" +++$+++ ")[-1])
    for i in range(len(ids)):
        qa_pairs = []
        
        if i==len(ids)-1:
            break
        
        first = remove_punc(lines_dic[ids[i]].strip())      
        second = remove_punc(lines_dic[ids[i+1]].strip())
        qa_pairs.append(first.split()[:max_len])
        qa_pairs.append(second.split()[:max_len])
        pairs.append(qa_pairs)

In [13]:
print(pairs[0])

[['can', 'we', 'make', 'this', 'quick', 'roxanne', 'korrine', 'and', 'andrew', 'barrett', 'are', 'having', 'an', 'incredibly', 'horrendous', 'public', 'break', 'up', 'on', 'the', 'quad', 'again'], ['well', 'i', 'thought', 'wed', 'start', 'with', 'pronunciation', 'if', 'thats', 'okay', 'with', 'you']]


In [14]:
word_freq = Counter()
for pair in pairs:
    word_freq.update(pair[0])
    word_freq.update(pair[1])

In [15]:
print(word_freq)

Counter({'you': 169695, 'i': 137639, 'the': 120903, 'to': 100667, 'a': 89110, 'it': 60011, 'and': 52131, 'of': 47393, 'what': 44771, 'that': 44755, 'in': 41692, 'me': 41056, 'is': 37374, 'dont': 33134, 'this': 30502, 'do': 30487, 'im': 30345, 'know': 29121, 'for': 29046, 'have': 27982, 'no': 27124, 'your': 26791, 'not': 26496, 'my': 26250, 'we': 25912, 'was': 25287, 'its': 24458, 'he': 24379, 'be': 24193, 'on': 23908, 'but': 22026, 'are': 21713, 'with': 21394, 'just': 20634, 'like': 19130, 'all': 18850, 'about': 18683, 'youre': 18180, 'get': 17562, 'if': 16727, 'so': 16697, 'well': 16263, 'out': 16100, 'here': 15848, 'they': 15365, 'got': 14745, 'want': 14744, 'thats': 14742, 'him': 14721, 'think': 14340, 'up': 14316, 'can': 14103, 'were': 14085, 'how': 14001, 'at': 13283, 'there': 12922, 'one': 12865, 'right': 12793, 'go': 12495, 'why': 12219, 'now': 12037, 'did': 11887, 'her': 11430, 'going': 11379, 'she': 11346, 'yes': 11203, 'oh': 10884, 'yeah': 10451, 'see': 10288, 'good': 9381, '

In [16]:
min_word_freq = 5
words = [w for w in word_freq.keys() if word_freq[w] > min_word_freq]
word_map = {k: v + 1 for v, k in enumerate(words)}
word_map['<unk>'] = len(word_map) + 1
word_map['<start>'] = len(word_map) + 1
word_map['<end>'] = len(word_map) + 1
word_map['<pad>'] = 0

In [17]:
print("Total words are: {}".format(len(word_map)))

Total words are: 18243


In [18]:
with open('WORDMAP_corpus.json', 'w') as j:
    json.dump(word_map, j)

In [19]:
def encode_question(words, word_map):
    print("question==",len(words))
    enc_c = [word_map.get(word, word_map['<unk>']) for word in words] + [word_map['<pad>']] * (max_len - len(words))
    print("encoded_question=",len(enc_c))
    return enc_c

In [20]:
def encode_reply(words, word_map):
    print("reply==",words)
    enc_c = [word_map['<start>']] + [word_map.get(word, word_map['<unk>']) for word in words] + \
    [word_map['<end>']] + [word_map['<pad>']] * (max_len - len(words))
    return enc_c

In [21]:
pairs_encoded = []
counter = 0
for pair in pairs:
    qus = encode_question(pair[0], word_map)
    ans = encode_reply(pair[1], word_map)
    pairs_encoded.append([qus, ans])
    counter = counter + 1
    if counter == 1:
        break

question== 22
encoded_question= 25
reply== ['well', 'i', 'thought', 'wed', 'start', 'with', 'pronunciation', 'if', 'thats', 'okay', 'with', 'you']


In [22]:
print(word_map['<unk>'])

18240


In [23]:
def create_masks(question, reply_input, reply_target):
    
    def subsequent_mask(size):
        mask = torch.triu(torch.ones(size, size)).transpose(0, 1).type(dtype=torch.uint8)
        return mask.unsqueeze(0)
    
    question_mask = question!=0
    question_mask = question_mask.to(device)
    question_mask = question_mask.unsqueeze(1).unsqueeze(1)         # (batch_size, 1, 1, max_words)
     
    reply_input_mask = reply_input!=0
    reply_input_mask = reply_input_mask.unsqueeze(1)  # (batch_size, 1, max_words)
    reply_input_mask = reply_input_mask & subsequent_mask(reply_input.size(-1)).type_as(reply_input_mask.data) 
    reply_input_mask = reply_input_mask.unsqueeze(1) # (batch_size, 1, max_words, max_words)
    reply_target_mask = reply_target!=0              # (batch_size, max_words)
    
    return question_mask, reply_input_mask, reply_target_mask